In [ ]:
import numpy as np
from numpy import random
import pandas as pd
from tqdm import tqdm
import multiprocessing as mp

# This notebooks purpose is to first generate a spin glass dataset of 9x9x9 and store them.
Additionally, this is meant to replace configMetropolis.py....
The lattices will undergo single_spin_flip updates as also parallel tempering over 10**5 montecarlo steps. 

In [ ]:
maxTemp = 1.5
temp_steps = np.arange(0.8,maxTemp,0.1)
MonteCarloSteps = 10**5
measureEvery = 1000
L = 4

num_disorder_configs = 1000  # The amount of disorder (aka bonds configured between the lattices) we want...

In [ ]:
def calculate_overlap(s, s_prime, n):
    """
    This function calculates the overlap between two spin configurations.
    The overlap is defined as the dot product of the spin vectors divided by the total number of spins.

    Parameters:
    s (aka lattice 1): The first spin configuration.
    s_prime (aka lattice 2): The second spin configuration.
    n: the dimension of the lattice (nxnxn) Ex:(if 3x3x3 then n=3)

    Returns:
    The overlap between the two spin configurations.
    """
    N = n ** 3 #the dimension n^3 because three dimensional
    s = np.array(s)
    s_prime = np.array(s_prime)
    q = np.sum(s * s_prime) / N
    return q

In [ ]:
# Initialize an empty dictionary
results = {temp: {'spins1': [], 'spins2': []} for temp in temp_steps}

In [ ]:
import multiprocessing as mp
from worker import worker

params = (L, MonteCarloSteps, temp_steps, measureEvery)
with mp.Pool(mp.cpu_count()) as pool:
    results_list = pool.map(worker, [params]*num_disorder_configs)

In [ ]:
for spins1, spins2 in results_list:
    for temp in temp_steps:
        results[temp]['spins1'].append(spins1[temp])
        results[temp]['spins2'].append(spins2[temp])

In [ ]:
results

In [ ]:
# import pickle

# Save the dictionary
# with open('spinconfigurations_L4.pkl', 'wb') as f:
#     pickle.dump(results, f)

# # Load the dictionary
# with open('spinconfigurations_L4.pkl', 'rb') as f:
#     loaded_results = pickle.load(f)

In [ ]:
overlap = {} #defining dictionary to hold the lists of overlap values at each temperature...

for temp in results:
    overlap[temp] = [calculate_overlap(s1, s2, 4) for s1, s2 in zip(results[temp]['spins1'], results[temp]['spins2'])]


In [ ]:
#overlap

In [ ]:
import matplotlib.pyplot as plt

# Assuming temperatures are sorted
temperatures = sorted(overlap.keys())

# Calculate the mean overlap for each temperature
mean_overlaps = [np.mean(overlap[temp]) for temp in temperatures]

plt.figure(figsize=(10, 6))
plt.plot(temperatures, mean_overlaps, marker='o')
plt.xlabel('Temperature')
plt.ylabel('Mean Overlap')
plt.title('Mean Overlap vs Temperature')
plt.grid(True)
plt.show()
